# Context Tests

## Setup

### Imports

In [ ]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Logging

In [ ]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Two Line SubSection Tests')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)

### Display Functions

In [ ]:
#%%writefile display_functions.py

In [ ]:
# %% Function to compare context for two sections.
def compare_context(section1, section2):
    ctx_template = '{key:16s}:\t{item1:16s}\t{item2:16s}'
    context_1 = section1.context
    context_2 = section2.context
    keys_1 = set(context_1.keys())
    keys_2 = set(context_2.keys())
    all_keys = keys_1 | keys_2
    for key in all_keys:
        item1 = context_1.get(key, '')
        item2 = context_2.get(key, '')
        ctx_str = ctx_template.format(key=str(key), item1=str(item1), item2=str(item2))
        print(ctx_str)
        
        
# %% Compare Buffered Iterator contents
def buffered_iterator_compare(iter1, iter2=None, iter3=None, 
                              label1='From Iterator', 
                              label2='To Iterator', label3=''):
    
    def extract_attrs(buf_obj, requested_item, as_list=True):
        if not buf_obj:
            text = ''
        elif as_list:
            text = str(list(buf_obj.__getattribute__(requested_item)))
        else:
            text = str(buf_obj.__getattribute__(requested_item))
        return text
        
    def extract_attr_text(requested_item, iter1, iter2=None, iter3=None, 
                        as_list=True):    
        attr_text = {
            1: extract_attrs(iter1, requested_item, as_list),
            2: extract_attrs(iter2, requested_item, as_list),
            3: extract_attrs(iter3, requested_item, as_list),
        }
        return attr_text


    row_template = ''.join([
        '\t{Label:<20s}',
        '{first_iter_item:<35s}',
        '{second_iter_item:<35s}',
        '{third_iter_item:<35s}\n'
        ])   
    attr_group = {
        'Previous Items': ('previous_items', True),
        'Future Items': ('future_items', True),
        'Item Count': ('item_count', False),
        'Step Back': ('_step_back', False),
        'Buffer Size': ('buffer_size', False)
        }

    row_list = [
        row_template.format(
            Label='',
            first_iter_item=label1, 
            second_iter_item=label2, 
            third_iter_item=label3)
                ]

    for label, attr_s in attr_group.items():
        requested_item, as_list = attr_s
        text_group = extract_attr_text(requested_item, iter1, iter2, iter3, as_list)
        text_line = row_template.format(Label=label, 
                        first_iter_item=text_group[1],
                        second_iter_item=text_group[2],
                        third_iter_item=text_group[3])
        row_list.append(text_line)
    
    iterator_compare_str = ''.join(row_list)
    
    return iterator_compare_str

## Check Context

***Context Standard Items***
<table>
<thead><th>Item Key</th><th>Value Description</th></thead>
<tr><td>Current Section</td>
<td>The assigned name of the section <i>self.section_name</i></td></tr>

<tr><td>Skipped Lines</td>
<td>A list of items in the source occurring before the first item of the section</td></tr>

<tr><td>Status</td>
<td>The current section scanning status one of:<ul>
    <li>Not Started
    <li>At section start
    <li>Scan In Progress
    <li>Break Triggered
    <li>Scan Complete
    <li>End of Source
    </ul></td></tr>
<tr><td>Break</td>
<td>The name assigned to the SectionBreak that most recently triggered. 
Before the section start this is not defined. 
While the section scan is in progress, this is the name of the StartSection. 
After the section ends, this is the name of the the EndSection.</td></tr>
<tr><td>
Event</td>
<td>The result from the SectionBreak that most recently triggered; the 
<i>TriggerEvent.test_value</i>.  
For more information see the TriggerEvent documentation.</td></tr>
</table>

- Compare `full_section` and `sub_section` ___Context___ after reading.


In [ ]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'Text between sections',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

In [ ]:
sub_section = Section(
    section_name='Sub-Section',
    start_section=SectionBreak('StartSection', break_offset='Before',
                               name='SubSectionStart'),
    end_section=SectionBreak('EndSection', break_offset='After', 
                             name='SubSectionEnd')
    )
full_section = Section(
    section_name='Top Section',
    end_section=SectionBreak('ignored', break_offset='Before', 
                             name='End Section'),
    processor=sub_section
    )
a = full_section.read(GENERIC_TEST_TEXT, context={'Dummy': 'Blank1'})
print()
print('full_section context')
pprint(full_section.context)
pprint(full_section.scan_status)
print()
print('sub_section context')
pprint(sub_section.context)
pprint(sub_section.scan_status)


full_section context
{'Break': 'End Section',
 'Current Section': 'Top Section',
 'Dummy': 'Blank1',
 'Event': 'ignored',
 'Skipped Lines': [],
 'Status': 'Break Triggered'}
'Break Triggered'

sub_section context
{'Break': 'SubSectionStart',
 'Current Section': 'Sub-Section',
 'Dummy': 'Blank1',
 'Event': 'StartSection',
 'Skipped Lines': ['Text between sections'],
 'Status': 'End of Source'}
'End of Source'


**Context Tests**
<style type="text/css">
.tg th{font-weight:bold;text-align:center;vertical-align:middle}
.tg td{font-weight:normal;text-align:left;vertical-align:top}
.rh td{font-weight:bold;text-align:center;vertical-align:middle}
</style>

<table class="tg">
    <thead>
        <tr>
            <th rowspan="2" text-align="center">Key</th>
            <th colspan="3">Top Section</th>
            <th colspan="3">Sub-Section</th>
        </tr><tr>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
            <th>Expected Value</th><th>Actual Value</th><th>Test</th>
        </tr>
    </thead>
        <tr><td class="rh">Current Section</td>
            <td>'Top Section'</td>
            <td>'Top Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
            <td>'Sub-Section'</td>
            <td>'Sub-Section'</td>
            <td><img src="../examples/Valid.png" alt="Good"/></td>
        </tr>
    <tr><td class="rh">Dummy</td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'Blank1'</td>  
        <td>'Blank1'</td>
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Skipped Lines</td>  
        <td>[]</td>
        <td>[]</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>['Text between sections']</td>
        <td>['Text between sections']</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Status</td>  
        <td>'Break Triggered'</td>
        <td>'Break Triggered'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'End of Source'</td>
        <td>'End of Source'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Break</td>  
        <td>'End Section'</td>
        <td>'End Section'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'SubSectionStart'</td>
        <td>'SubSectionStart'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
    <tr><td class="rh">Event</td>  
        <td>'ignored'</td>
        <td>'ignored'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
        <td>'StartSection'</td>
        <td>'StartSection'</td>            
        <td><img src="../examples/Valid.png" alt="Good"/></td>
    </tr>
</table>

- ***Current Section*** is the name of the section given at the section definition.
    - For _full_section_,  `name='End Section'`
    - For _sub_section_,   `name='Sub-Section'`
- ***Dummy*** is a context element supplied at the time `full_section.read()` was 
  called. Since it is not one of the standard context items it is just along for 
  the ride unless a user defined function explicitly modifies it. 
    - The initially supplied value of ***Dummy*** = _'Blank1'_ is the same for 
      both _full_section_ and _sub_section_.
- ***Skipped Lines*** is the list of unused lines encountered before the beginning 
  of the section.
    - For _full_section_, start_section is not given so ***Skipped Lines*** is 
      an empty list.
    - For _sub_section_, ***Skipped Lines*** is the lines skipped before the 
      most recent sub-section: ['Text between sections']
- ***Status*** is the section state. 
    - _full_section_ has ended because the `end_section` break has triggered, so
      ***Status*** = 'Break Triggered'
    - _sub_section_ receives it's source from _full_section_, but 
      _full_section_ has ended.  _sub_section_ interprets this as and end to 
      it's source, so ***Status*** = 'End of Source'
- ***Break*** is the name of the most recent break triggered while the section is 
   active.
    - The `end_section` break of _full_section_ has triggered.  The name of 
      this SectionBreak was given as 'End Section', so 
      ***Break*** = 'End Section'.
    - The `end_section` break of _sub_section_ never triggered because it's 
      source ended first. The most recent section break for _sub_section_ was 
      the `start_section`, so ***Break*** = 'SubSectionStart'.
- ***Event*** is the Trigger specific value resulting from the last break.  For 
  string based breaks, ***Event*** is the string used to test for the break.
    - The `end_section` break of _full_section_ was triggered by the string 
      'ignored', so ***Event*** = 'ignored'.
    - The `start_section` break of _sub_section_ was triggered by the string 
      'StartSection', so ***Event*** = 'StartSection'.